### Imports

In [1]:
import itertools as it
import string
import pandas as pd
from datetime import datetime

### Data

In [2]:
pcdf = pd.read_csv('Work_tables/gcs_results.csv').dropna(subset=['ACTIVE'])

In [3]:
excel_lookup = pd.ExcelFile('Work_tables/GCS_Lookup_Table.xlsx')

### Combine geocoder results and lookup tables

In [4]:
pcdf = pcdf.loc[pcdf['ACTIVE'] == 'Y']

In [5]:
pcdf = pcdf.drop('ACTIVE', axis=1)

In [6]:
sheet_names = excel_lookup.sheet_names

In [7]:
common_sheets = list(set(pcdf.columns).intersection(sheet_names))

In [8]:
lookup_dict = {}

In [9]:
for s in common_sheets:

    lookup_dict[s] =  {'file': pd.read_excel('Work_tables/GCS_Lookup_Table.xlsx', sheet_name = s)}
    lookup_dict[s]['key'] = ''.join(set(lookup_dict[s]['file'].columns.to_list()).intersection(set(pcdf.columns.to_list())))
    lookup_dict[s]['values'] = lookup_dict[s]['file'].columns.to_list()
    lookup_dict[s]['values'].remove(lookup_dict[s]['key'])
    numeric_lookup_cols = lookup_dict[s]['file'].select_dtypes(include='number').columns.tolist()
    lookup_dict[s]['file'][numeric_lookup_cols] = lookup_dict[s]['file'][numeric_lookup_cols].astype('float')

Join geocoder results with the look up table

In [10]:
for sheet in common_sheets:
    pcdf = pcdf.merge(lookup_dict[sheet]['file'], how = 'left')

In [11]:
for sheet in common_sheets:
    for i in range(len(lookup_dict[sheet]['values'])):

        no_match_count = (pcdf[lookup_dict[sheet]['file']
             .columns.to_list()]
             .loc[(pcdf[lookup_dict[sheet]['key']].isna() == False) & pcdf[lookup_dict[sheet]['values'][i]]
             .isna()]
             .shape[0]
            )
        
        if no_match_count != 0:
            print(f"{sheet}: {lookup_dict[sheet]['values'][i]}. Count: {no_match_count}")
        else:
            print(f"{sheet}: {lookup_dict[sheet]['values'][i]}. No issues")

CDCSD_2021: CDCSDNAME. No issues
CDCSD_2021: CDCSDTYPE. No issues
DPL_2021: DPLNAME. No issues
CMACA_2021: CMANAME. No issues
POPCTR_2021: PCNAME. No issues
CD_2021: CDNAME. No issues
DR_2021: DRNAME. No issues


In [12]:
for sheet in lookup_dict:
    new_loc = pcdf.columns.get_loc(lookup_dict[sheet]['key']) + 1
    for i in range(len(lookup_dict[sheet]['values'])):
        col_to_move = pcdf.pop(lookup_dict[sheet]['values'][i])
        pcdf.insert(new_loc, lookup_dict[sheet]['values'][i], col_to_move)

In [14]:
pcdf.to_excel(''.join(
    ['_'.join(
        ['Geocoded_datasets/Geocoded_postal_codes', 
         datetime.today().strftime('%Y_%m_%d_time_%H_%M')]
    ), '.xlsx']),
    index=False
    )